    ## Activity based classifier

In [2]:
import graphlab as gl
login = gl.SFrame('data/user_login_info.csv')
print login

[INFO] This commercial license of GraphLab Create is assigned to mjdata@mindjet.com.

[INFO] Start server at: ipc:///tmp/graphlab_server-6875 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1440468637.log
[INFO] GraphLab Server Version: 1.5.2


PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_login_info.csv
PROGRESS: Parsing completed. Parsed 100 lines in 4.53704 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 3172097 lines. Lines per second: 775333
PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_login_info.csv
PROGRESS: Parsing completed. Parsed 4000000 lines in 4.46445 secs.
+---------+------------+----------------+-------------+----------+
| user_id | country_id | primary_device | days_active | activity |
+---------+------------+----------------+-------------+----------+
|    1    |     h      |       B   

Split dataset into 80/20 training and test set

In [32]:
train_login, test_login = login.random_split(0.8, seed=1)

Create a (binary or multi-class) classifier model of type BoostedTreesClassifier using gradient boosted trees.

In [33]:
features = ['country_id', 'primary_device', 'days_active']
model_login = gl.boosted_trees_classifier.create(
            train_login,
            target='activity',
            features=features,
            max_iterations=10)

model_login

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 3040523
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 3
PROGRESS: Number of unpacked features : 3
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.233e-01   8.230e-01        2.54s
PROGRESS:      1   8.246e-01   8.243e-01        4.92s
PROGRESS:      2   8.243e-01   8.241e-01        7.49s
PROGRESS:      3   8.247e-01   8.245e-01        9.85s
PROGRESS:      4   8.246e-01   8.244e-01       12.23s
PROGRESS:      5   8.248e-01   8.243e-01       14.69s
PROGRESS:      6   8.248e-01   8.

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 3040523
Number of classes             : 2
Number of feature columns     : 3
Number of unpacked features   : 3

Settings
--------
Number of trees               : 10
Max tree depth                : 6
Train accuracy                : 0.825
Validation accuracy           : 0.8246
Training time (sec)           : 28.8072

Evalue with test dataset

In [34]:
res_login = model_login.evaluate(test_login)
res_login

{'accuracy': 0.8248556344456588, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      f       |        f        | 44735  |
 |      t       |        f        | 28345  |
 |      t       |        t        | 614907 |
 |      f       |        t        | 111719 |
 +--------------+-----------------+--------+
 [4 rows x 3 columns]}

There are a lot of false positives. Many people who are actually inactive are being classified as active. Lets see if we can improve the classifier accuracy by bringing in some more activity based data from other source.

In [3]:
eng = gl.SFrame.read_csv('data/user_engagement.csv')
login_eng = login.join(eng, on='user_id', how='inner')
print login_eng

PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_engagement.csv
PROGRESS: Parsing completed. Parsed 100 lines in 1.52739 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 1737129 lines. Lines per second: 557364
PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_engagement.csv
PROGRESS: Parsing completed. Parsed 4000000 lines in 4.31871 secs.
+---------+------------+----------------+-------------+----------+----------------+
| user_id | country_id | primary_device | days_active | activity | num_tweets_30d |
+---------+------------+----------------+-----------

In [45]:
train_login_eng, test_login_eng = login_eng.random_split(0.8, seed=1)

Remodel classifier with entire dataset

In [46]:
features=[
    'country_id',
    'primary_device',
    'days_active',
    'num_tweets_30d',
    'num_tweet_days_30d',
    'time_in_app_30d',
    'num_timeline_views_30d',
    'num_share_sent_days_30d',
    'num_share_rcvd_30d',
    'num_favour_sent_30d',
    'num_favour_rcvd_30d'                
]

model_login_eng = gl.boosted_trees_classifier.create(
            train_login_eng,
            target='activity',
            features=features,
            max_iterations=10)

model_login_eng

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 3040444
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 11
PROGRESS: Number of unpacked features : 11
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.888e-01   8.890e-01       16.64s
PROGRESS:      1   8.895e-01   8.896e-01       33.47s
PROGRESS:      2   8.898e-01   8.897e-01       49.80s
PROGRESS:      3   8.899e-01   8.899e-01       66.29s
PROGRESS:      4   8.900e-01   8.901e-01       85.05s
PROGRESS:      5   8.901e-01   8.901e-01      103.40s
PROGRESS:      6   8.901e-01   

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 3040444
Number of classes             : 2
Number of feature columns     : 11
Number of unpacked features   : 11

Settings
--------
Number of trees               : 10
Max tree depth                : 6
Train accuracy                : 0.8903
Validation accuracy           : 0.8904
Training time (sec)           : 174.9902

In [47]:
res_login_eng = model_login_eng.evaluate(test_login_eng)
res_login_eng

{'accuracy': 0.8896131828446954, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      f       |        t        | 43870  |
 |      f       |        f        | 112584 |
 |      t       |        f        | 44407  |
 |      t       |        t        | 598845 |
 +--------------+-----------------+--------+
 [4 rows x 3 columns]}

The accuracy improved from 0.82 to 0.88. This means it is a good idea to include the engagement data for training an activity classifier.

Let us now investigate if incorporating graph info into the training data increases accuracy.

In [4]:
graph = gl.SFrame.read_csv('data/user_graph.csv')
login_eng_graph = login_eng.join(graph, on='user_id', how='inner')
login_eng_graph = login_eng_graph.dropna()
login_eng_graph = login_eng_graph.fillna('followers', 0).fillna('followings',0).fillna('active_followers',0).fillna('num_mutual_follower_added_30d', 0)
print login_eng_graph

PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_graph.csv
PROGRESS: Parsing completed. Parsed 100 lines in 1.53772 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 2641306 lines. Lines per second: 1.24937e+06
PROGRESS: Finished parsing file /Users/mhardas/Google Drive/work/project/twitter-data-challenge/data/user_graph.csv
PROGRESS: Parsing completed. Parsed 4000000 lines in 2.49864 secs.
+---------+------------+----------------+-------------+----------+----------------+
| user_id | country_id | primary_device | days_active | activity | num_tweets_30d |
+---------+------------+----------------+-------------+----------+-------

In [57]:
features=[
    'country_id',
    'primary_device',
    'days_active',
    'num_tweets_30d',
    'num_tweet_days_30d',
    'time_in_app_30d',
    'num_timeline_views_30d',
    'num_share_sent_days_30d',
    'num_share_rcvd_30d',
    'num_favour_sent_30d',
    'num_favour_rcvd_30d',
    'followers',
    'followings',
    'active_followers',
    'num_mutual_follower_added_30d'
    ]

train_login_eng_graph, test_login_eng_graph = login_eng_graph.random_split(0.8, seed=1)
model_login_eng_graph = gl.boosted_trees_classifier.create(
            train_login_eng_graph,
            target='activity',
            features=features,
            max_iterations=10)

model_login_eng_graph

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2979713
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 15
PROGRESS: Number of unpacked features : 15
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.920e-01   8.917e-01       26.26s
PROGRESS:      1   8.930e-01   8.934e-01       52.25s
PROGRESS:      2   8.932e-01   8.936e-01       77.23s
PROGRESS:      3   8.933e-01   8.934e-01      102.58s
PROGRESS:      4   8.934e-01   8.935e-01      126.54s
PROGRESS:      5   8.935e-01   8.936e-01      151.16s
PROGRESS:      6   8.939e-01   

Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 2979713
Number of classes             : 2
Number of feature columns     : 15
Number of unpacked features   : 15

Settings
--------
Number of trees               : 10
Max tree depth                : 6
Train accuracy                : 0.894
Validation accuracy           : 0.8941
Training time (sec)           : 257.1845

In [58]:
res_login_eng_graph = model_login_eng_graph.evaluate(test_login_eng_graph)
res_login_eng_graph

{'accuracy': 0.8939041978838275, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      f       |        f        | 102000 |
 |      f       |        t        | 42482  |
 |      t       |        f        | 40643  |
 |      t       |        t        | 598365 |
 +--------------+-----------------+--------+
 [4 rows x 3 columns]}

We can see that the accuracy marginally increased from 0.88 tp 0.894. It is very small improvement which may or may not be attributed to user graph data. Let us try to figure out if we can further increase the accuracy.

###Basic feature selection

It seems that engagement metrics are the real informatics metrics about activity. So we drop all features from user login info and user graph and select on the engagement metrics to build the model.

Feature hasher is a method in the graphlab package that we can use to engineer useful predictor metrics from existing engagement metrics to see if they are more informative. Basic idea of feature hashing is to collaps a feature vector by reducing the dimensionality.
https://dato.com/products/create/docs/generated/graphlab.toolkits.feature_engineering._feature_hasher.FeatureHasher.html#graphlab.toolkits.feature_engineering._feature_hasher.FeatureHasher

In [6]:
from graphlab import feature_engineering as fe

simple_features=[
    'num_tweets_30d',
    'num_tweet_days_30d',
    'time_in_app_30d',
    'num_timeline_views_30d',
    'num_share_sent_days_30d',
    'num_share_rcvd_30d',
    'num_favour_sent_30d',
    'num_favour_rcvd_30d',
    ]

train_login_eng_graph, test_login_eng_graph = login_eng_graph.random_split(0.8, seed=12345)

hasher = fe.create(login_eng_graph, fe.FeatureHasher(features=simple_features))
hashed_train_login_eng_graph = hasher.transform(train_login_eng_graph)
hashed_test_login_eng_graph = hasher.transform(test_login_eng_graph)
training_features = simple_features.append('hashed_features')

model_login_eng_graph = gl.boosted_trees_classifier.create(
            hashed_train_login_eng_graph,
            target='activity',
            features=training_features,
            max_iterations=10)

res_login_eng_graph = model_login_eng_graph.evaluate(hashed_test_login_eng_graph)
res_login_eng_graph

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2978510
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 9
PROGRESS: Number of unpacked features : 16
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.921e-01   8.918e-01       27.22s
PROGRESS:      1   8.922e-01   8.920e-01       52.87s
PROGRESS:      2   8.930e-01   8.927e-01       78.01s
PROGRESS:      3   8.932e-01   8.933e-01      101.72s
PROGRESS:      4   8.933e-01   8.936e-01      125.20s
PROGRESS:      5   8.933e-01   8.938e-01      148.90s
PROGRESS:      6   8.936e-01   8

{'accuracy': 0.8939557233742773, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      f       |        t        | 42427  |
 |      f       |        f        | 102728 |
 |      t       |        f        | 40815  |
 |      t       |        t        | 599004 |
 +--------------+-----------------+--------+
 [4 rows x 3 columns]}

The accuracy marginally increased to 0.894. The false positives decreased to 42427 from 42484.

In [8]:
from graphlab import feature_engineering as fe

simple_features=[
    'num_tweets_30d',
    'num_tweet_days_30d',
    'time_in_app_30d',
    'num_timeline_views_30d',
    'num_share_sent_days_30d',
    'num_share_rcvd_30d',
    'num_favour_sent_30d',
    'num_favour_rcvd_30d',
    ]

train_login_eng_graph, test_login_eng_graph = login_eng_graph.random_split(0.8, seed=12345)

quad = fe.create(login_eng_graph, fe.FeatureHasher(features=simple_features))
quad_train_login_eng_graph = quad.transform(train_login_eng_graph)
quad_test_login_eng_graph = quad.transform(test_login_eng_graph)
training_features = simple_features.append('quadratic_features')

model_login_eng_graph_quad = gl.boosted_trees_classifier.create(
            quad_train_login_eng_graph,
            target='activity',
            features=training_features,
            max_iterations=10)

res_login_eng_graph_quad = model_login_eng_graph_quad.evaluate(quad_test_login_eng_graph)
res_login_eng_graph_quad

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 2978459
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 9
PROGRESS: Number of unpacked features : 16
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   8.921e-01   8.917e-01       24.87s
PROGRESS:      1   8.928e-01   8.923e-01       49.47s
PROGRESS:      2   8.925e-01   8.921e-01       73.89s
PROGRESS:      3   8.932e-01   8.924e-01       98.32s
PROGRESS:      4   8.933e-01   8.925e-01      122.70s
PROGRESS:      5   8.935e-01   8.928e-01      147.17s
PROGRESS:      6   8.936e-01   8

{'accuracy': 0.8939226012581308, 'confusion_matrix': Columns:
 	target_label	str
 	predicted_label	str
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      f       |        t        | 41800  |
 |      f       |        f        | 103355 |
 |      t       |        f        | 41468  |
 |      t       |        t        | 598351 |
 +--------------+-----------------+--------+
 [4 rows x 3 columns]}

Quadratic Features is another method in the graphlab package that we can use to engineer useful predictor metrics from existing engagement metrics to see if they are more informative. Basic idea of quadratic features is to compute a feature which is the combination of all training features.

The number of false positive went down from 42427 to 41800 by using quadratic features.